In [6]:
import pandas as pd
import mysql.connector
from mysql.connector import errorcode


In [7]:
# ✅ 1️⃣ Connect to MySQL (root or admin user first)
try:
    root_conn = mysql.connector.connect(
        host="localhost",
        user="newuser",
        password="password123"  # change this!
    )
    root_cursor = root_conn.cursor()
    print("✅ Connected to MySQL successfully.")
except mysql.connector.Error as err:
    print("❌ Error connecting to MySQL:", err)
    raise



✅ Connected to MySQL successfully.


In [8]:
# ✅ 2️⃣ Create database if it doesn't exist
db_name = "mexico_toy_store_wh_db"
root_cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
print(f"✅ Database '{db_name}' created (or already exists).")

✅ Database 'mexico_toy_store_wh_db' created (or already exists).


In [9]:
root_cursor.execute(f"USE {db_name}")

In [10]:
#Dataframes
calendar_df=pd.read_csv("~/Desktop/Group_AI/IndusAnalProj/data/calendar.csv")

inventory_df=pd.read_csv("~/Desktop/Group_AI/IndusAnalProj/data/inventory.csv")
products_df=pd.read_csv("~/Desktop/Group_AI/IndusAnalProj/data/products.csv")
sales_df=pd.read_csv("~/Desktop/Group_AI/IndusAnalProj/data/sales.csv", low_memory=False)
store_df=pd.read_csv("~/Desktop/Group_AI/IndusAnalProj/data/stores.csv")

In [11]:
calendar_df.head()

,Date
0,1/1/2022
1,1/2/2022
2,1/3/2022
3,1/4/2022
4,1/5/2022


In [12]:
store_df.head()

,Store_ID,Store_Name,Store_City,Store_Location,Store_Open_Date
0,1,Maven Toys Guadalajara 1,Guadalajara,Residential,1992-09-18
1,2,Maven Toys Monterrey 1,Monterrey,Residential,1995-04-27
2,3,Maven Toys Guadalajara 2,Guadalajara,Commercial,1999-12-27
3,4,Maven Toys Saltillo 1,Saltillo,Downtown,2000-01-01
4,5,Maven Toys La Paz 1,La Paz,Downtown,2001-05-31


In [13]:
inventory_df.head()

,Store_ID,Product_ID,Stock_On_Hand
0,1,1,27
1,1,2,0
2,1,3,32
3,1,4,6
4,1,5,0


In [14]:
products_df.head()

,Product_ID,Product_Name,Product_Category,Product_Cost,Product_Price
0,1,Action Figure,Toys,$9.99,$15.99
1,2,Animal Figures,Toys,$9.99,$12.99
2,3,Barrel O' Slime,Art & Crafts,$1.99,$3.99
3,4,Chutes & Ladders,Games,$9.99,$12.99
4,5,Classic Dominoes,Games,$7.99,$9.99


In [15]:
sales_df.head()

,Sale_ID,Date,Store_ID,Product_ID,Units
0,1,2022-01-01,24,4,1
1,2,2022-01-01,28,1,1
2,3,2022-01-01,6,8,1
3,4,2022-01-01,48,7,1
4,5,2022-01-01,44,18,1


In [16]:
root_cursor.close()
root_conn.close()

# ✅ 3️⃣ Reconnect — this time directly to africa_wh_db
conn = mysql.connector.connect(
    host="localhost",
    user="newuser",
    password="password123",
    database=db_name
)
cur = conn.cursor()

# ✅ 4️⃣ Define all your tables and dataframes
tables = [
    {
        "name": "calendars_tbl",
        "data": calendar_df,
        "columns": calendar_df.columns
    },
    {
        "name": "inventory_tbl",
        "data": inventory_df,
        "columns": inventory_df.columns
    },
    {
        "name": "products_tbl",
        "data": products_df,
        "columns": products_df.columns
    },
    {
        "name": "sales_tbl",
        "data": sales_df,
        "columns": sales_df.columns
    },
    {
        "name": "stores_tbl",
        "data": store_df,
        "columns": store_df.columns
    }
]

In [17]:
# ✅ 5️⃣ Loop through tables to create and insert data
for t in tables:
    table_name = t["name"]
    df = t["data"].copy()
    columns = t["columns"]
    df.columns = columns  

    # --- CREATE TABLE ---
    create_query = f"""
    CREATE TABLE IF NOT EXISTS {table_name} (
        {', '.join([f'`{col}` TEXT' for col in columns])}
    );
    """

    try:
        cur.execute(create_query)
        conn.commit()
        print(f"✅ Table '{table_name}' created successfully.")
    except mysql.connector.Error as err:
        print(f"❌ Error creating table '{table_name}': {err}")
        conn.rollback()
        continue

    # --- INSERT DATA ---
    try:
        placeholders = ','.join(['%s'] * len(columns))
        insert_query = f"""
            INSERT INTO {table_name} ({', '.join([f'`{col}`' for col in columns])})
            VALUES ({placeholders})
        """

        # Use executemany for speed
        cur.executemany(
            insert_query,
            [tuple(str(row[c]) for c in columns) for _, row in df.iterrows()]
        )

        conn.commit()
        print(f"✅ Data inserted successfully into '{table_name}'.\n")
    except mysql.connector.Error as err:
        print(f"❌ Error inserting data into '{table_name}': {err}")
        conn.rollback()

cur.close()
conn.close()
print("🏁 All tables processed successfully.")

✅ Table 'calendars_tbl' created successfully.
✅ Data inserted successfully into 'calendars_tbl'.

✅ Table 'inventory_tbl' created successfully.
✅ Data inserted successfully into 'inventory_tbl'.

✅ Table 'products_tbl' created successfully.
✅ Data inserted successfully into 'products_tbl'.

✅ Table 'sales_tbl' created successfully.
✅ Data inserted successfully into 'sales_tbl'.

✅ Table 'stores_tbl' created successfully.
✅ Data inserted successfully into 'stores_tbl'.

🏁 All tables processed successfully.
